<a href="https://colab.research.google.com/github/khushiyadav03/Machine-Learning/blob/main/Feature_extraction_of_Text_data_using_Tf_idf_Vectorizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article

2. title: the title of a news article

3. author: author of the news article

4. text: the text of the article; could be incomplete

5. label: a label that marks whether the news article is real or fake:
* 1: Fake news
* 0: real News

In [63]:
import numpy as np # Data ko numbers mein handle karne ke liye (for handling data in numbers)
import pandas as pd # DataFrames (tables) banane aur un par kaam karne ke liye (for creating and working with DataFrames/tables)
import re # Text se patterns dhoondhne aur badalne ke liye (for finding and replacing patterns in text)
from nltk.corpus import stopwords # Stopwords (jaise 'the', 'a', 'is') ko use karne ke liye (for using stopwords like 'the', 'a', 'is')
from nltk.stem.porter import PorterStemmer # Words ko unki root form mein lane ke liye (like 'acting' to 'act') (for bringing words to their root form)
from sklearn.feature_extraction.text import TfidfVectorizer # Text data ko numerical features mein badalne ke liye (for converting text data into numerical features)
import nltk # Natural Language Processing (NLP) tasks ke liye (for Natural Language Processing tasks)

In [64]:
import nltk
# Stopwords download karna (Downloading stopwords)
# Ye ek baar hi karna hota hai (This needs to be done only once)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
# English language ke stopwords print karna (Printing the stopwords in English language)
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Pre-processing

In [37]:
# loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('/content/fake_news_dataset.csv', encoding='latin-1', engine='python', on_bad_lines='skip')

In [38]:
news_dataset.shape

(18383, 5)

In [39]:
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [40]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,490
author,1747
text,49
label,16


In [41]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [42]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [43]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didnât Even...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
18378    Mekado Murphy Holiday Movies to See (or Skip) ...
18379     Regretful Brexiter realises he actually just ...
18380    Trent Baker Sessions: The Border Is Not Open -...
18381    Alissa J. Rubin France Lets Workers Turn Off, ...
18382    Lucas Nolan REPORT: Ann Coulter Cancels UC Ber...
Name: content, Length: 18383, dtype: object


In [44]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [45]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didnât Even See Comeyâs...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
18378  18363  Holiday Movies to See (or Skip) - The New York...   
18379  18364  Regretful Brexiter realises he actually just r...   
18380  18365       Sessions: The Border Is Not Open - Breitbart   
18381  18366  France Lets Workers Turn Off, Tune Out and Liv...   
18382  18367  REPORT: Ann Coulter Cancels UC Berkeley Speech...   

                   author                                               text  \
0           Darrell Lucus  House Dem Aide: We Didnât Even See Comeyâs...   
1         Daniel J. Flynn  Ever get

In [46]:
print(Y)

0        1.0
1        0.0
2        1.0
3        1.0
4        1.0
        ... 
18378    0.0
18379    1.0
18380    0.0
18381    0.0
18382    0.0
Name: label, Length: 18383, dtype: object


Stemming:

Stemming is the process of reducing a word to its Root word

example: actor, actress, acting --> act

In [47]:
port_stem = PorterStemmer()

In [49]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [50]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [51]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
18378    mekado murphi holiday movi see skip new york time
18379    regret brexit realis actual realli miss old to...
18380            trent baker session border open breitbart
18381    alissa j rubin franc let worker turn tune live...
18382    luca nolan report ann coulter cancel uc berkel...
Name: content, Length: 18383, dtype: object


In [52]:
# seperating tha data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [53]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'trent baker session border open breitbart'
 'alissa j rubin franc let worker turn tune live life new york time'
 'luca nolan report ann coulter cancel uc berkeley speech breitbart']


In [54]:
print(Y)

[1.0 0.0 1.0 ... 0.0 0.0 0.0]


In [66]:
# 'Y' (labels) ka shape check karna (Checking the shape of 'Y' (labels))
Y.shape

(18383,)

Tf-ldf

In [67]:
# Textual data ko numerical feature vectors mein badalne ke liye TfidfVectorizer shuru karna (Initializing TfidfVectorizer to convert textual data into numerical feature vectors)
vectorizer = TfidfVectorizer()

In [68]:
# 'X' ko raw text data par set karna, vectorizer fit karne se pehle (Setting 'X' to raw text data before fitting the vectorizer)
X = news_dataset['content'].values

# Vectorizer ko data par fit karna (Fitting the vectorizer on the data)
vectorizer.fit(X)

# Data ko numerical feature vectors mein badalna (Transforming the data into numerical feature vectors)
X = vectorizer.transform(X)

In [69]:
# Numerical feature vectors 'X' ko print karna (Printing the numerical feature vectors 'X')
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 186206 stored elements and shape (18383, 16280)>
  Coords	Values
  (0, 258)	0.2669559997176808
  (0, 2383)	0.3648637143460868
  (0, 2846)	0.24608969923241478
  (0, 3454)	0.3698480218086287
  (0, 3640)	0.26789643180876965
  (0, 4753)	0.23279459057435217
  (0, 6692)	0.21894286803889415
  (0, 7340)	0.24524612905665602
  (0, 8225)	0.28904057017439516
  (0, 8483)	0.3698480218086287
  (0, 12808)	0.253861969074018
  (0, 14925)	0.28185083328012384
  (1, 1437)	0.2913881517919056
  (1, 1818)	0.1540565449050519
  (1, 2133)	0.3806443924373733
  (1, 2701)	0.19001149724844194
  (1, 3424)	0.2635109716821854
  (1, 5265)	0.7181626606825728
  (1, 6511)	0.18938262686832083
  (1, 15970)	0.2988370536454543
  (2, 2830)	0.318781975899352
  (2, 2984)	0.4657211114208176
  (2, 5156)	0.3844081325093558
  (2, 5702)	0.347429865210207
  (2, 9154)	0.4878706063465923
  :	:
  (18380, 10196)	0.42202652874654506
  (18380, 12906)	0.4738776824658882
  (18380, 1